In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../src/')
import wave as w
import utils as ut
import plotter
import numba_functions as nbf
import numba as nb
from copy import deepcopy
from scipy.optimize import brentq

## Plotting the speed and R0 at different D

In [2]:
def build_assay_wave(id_, r0, D, beta, gamma=1.0, cutoff=1):
    dx = ut.lin_from_two_points(np.log(r0/Delta_x), np.log(0.05), 0.01, np.log(5), 0.02)
    dt = ut.dt_from_cfl(0.03, D, dx)

    p = w.Vwave_pars(id_, tot_time=400, dt=dt, dx=dx, 
                     n_x_bins=int( ut.lin_from_two_points(r0/Delta_x, 0.05, 10, 5, 200)/dx ), 
                     M=5, r0=r0, D_coef=D, beta=beta, alpha=0.0, gamma=gamma,
                     Nh=10**10, N0=10**9, cutoff=cutoff,
                     t_burn=ut.lin_from_two_points(r0, 0.01, 200, 1, 6000), 
                     t_burn_cutoff=ut.lin_from_two_points(r0, 0.01, 70, 1, 1000), 
                     back_width_fract=ut.lin_from_two_points(np.log(r0/Delta_x), np.log(0.05), 0.5, np.log(5), 3),
                     traj_step=int(5/dt), check_step=int(100/dt), traj_after_burn=True, verbose=True)
    return w.Vwave(p)

## FKPP regime

In [3]:
r0 = 0.10345690405573947
Delta_x = 1
lambd = 250
lambda_tilde = 2*lambd/Delta_x**2

b_zero = 2
D_list = np.linspace(4,6,20)*10**(-4)
b_list = (1 - lambda_tilde * D_list) * b_zero

In [4]:
assays = []
for i in range(len(D_list)):
    assays.append(build_assay_wave(i, r0, D_list[i], b_list[i]))

proc. 0 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 1 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 2 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 3 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 4 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 5 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 6 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 7 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 8 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 9 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 10 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 11 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 12 WARNING: dx is not small compared to r0. dx/r0= 0.11192048973534587
proc. 13 

In [5]:
waves = ut.multiprocess_sim(assays, 4)

Process 2:   0%|          | 0/120126 [00:00<?, ?it/s]

Process 0:   0%|          | 0/114120 [00:00<?, ?it/s]

Process 1:   0%|          | 0/117123 [00:00<?, ?it/s]

Process 3:   0%|          | 0/123129 [00:00<?, ?it/s]

Process 4:   0%|          | 0/126132 [00:00<?, ?it/s]

...

Process 15:   0%|          | 0/159167 [00:00<?, ?it/s]

Process 16:   0%|          | 0/162170 [00:00<?, ?it/s]

Process 17:   0%|          | 0/165174 [00:00<?, ?it/s]

Process 18:   0%|          | 0/168177 [00:00<?, ?it/s]

Process 19:   0%|          | 0/171180 [00:00<?, ?it/s]

In [6]:
speeds = []
for i in range(len(D_list)):
    speeds.append(np.mean(waves[i].traj.speed(50)[-50:]))

In [11]:
f = open('data/speed_R0_vs_D_FKPP.tsv', 'w')

header = '#D\tspeed\tR0\n'
f.write(header)

for i in range(len(D_list)):
    f.write(str(D_list[i]) + '\t')
    f.write(str(speeds[i]) + '\t')
    f.write(str(b_zero*np.exp(-lambd*D_list[i])) + '\t')
    f.write('\n')
    
f.close()


f = open('data/speed_R0_vs_D_FKPP_pars.tsv', 'w')

header = '#parameter\tvalue\n'
f.write(header)

f.write('lambda\t' + str(lambda_tilde) + '\n')
f.write('beta0\t' + str(waves[0].p.beta) + '\n')
f.write('alpha\t' + str(waves[0].p.alpha) + '\n')
f.write('gamma\t' + str(waves[0].p.gamma) + '\n')
f.write('Nh\t' + str(waves[0].p.Nh) + '\n')
f.write('M\t' + str(waves[0].p.M) + '\n')
f.write('cutoff\t' + str(waves[0].p.cutoff) + '\n')

f.close()

## linear-fitness regime

In [12]:
def build_assay_wave(id_, r0, D, beta, gamma=1.0, cutoff=1):
    dx = ut.lin_from_two_points(np.log(r0/Delta_x), np.log(0.05), 0.025, np.log(7), 0.04)
    dt = ut.dt_from_cfl(0.03, D, dx)

    p = w.Vwave_pars(id_, tot_time=400, dt=dt, dx=dx, 
                     n_x_bins=int( ut.lin_from_two_points(r0/Delta_x, 0.05, 10, 7, 300)/dx ), 
                     M=5, r0=r0, D_coef=D, beta=beta, alpha=0.0, gamma=gamma,
                     Nh=10**10, N0=10**9, cutoff=cutoff,
                     t_burn=ut.lin_from_two_points(r0, 0.01, 200, 1, 10000), 
                     t_burn_cutoff=ut.lin_from_two_points(r0, 0.01, 70, 1, 1000), 
                     back_width_fract=ut.lin_from_two_points(np.log(r0/Delta_x), np.log(0.05), 0.5, np.log(5), 3),
                     traj_step=int(5/dt), check_step=int(0/dt), traj_after_burn=True, verbose=True)
    return w.Vwave(p)

In [13]:
r0 = 6.5
Delta_x = 1
lambd = 250
lambda_tilde = 2*lambd/Delta_x**2

b_zero = 2
D_list2 = np.linspace(1, 3, 15)*10**(-4)
b_list2 = (1 - lambda_tilde * D_list2)*b_zero

In [14]:
assays = []
for i in range(len(D_list2)):
    assays.append(build_assay_wave(i, r0, D_list2[i], b_list2[i]))

In [ ]:
waves2 = ut.multiprocess_sim(assays, 4)
#assays[0].run()

Process 0:   0%|          | 0/136624 [00:00<?, ?it/s]

Process 1:   0%|          | 0/156142 [00:00<?, ?it/s]

Process 2:   0%|          | 0/175660 [00:00<?, ?it/s]

Process 3:   0%|          | 0/195178 [00:00<?, ?it/s]

Process 4:   0%|          | 0/214696 [00:00<?, ?it/s]

Process 5:   0%|          | 0/234214 [00:00<?, ?it/s]

In [ ]:
speeds2 = []
for i in range(len(D_list2)):
    speeds2.append(np.mean(waves2[i].traj.speed(50)[-20:]))

In [ ]:
f = open('data/speed_R0_vs_D_linfit.tsv', 'w')

header = '#D\tspeed\tR0\n'
f.write(header)

for i in range(len(D_list)):
    f.write(str(D_list2[i]) + '\t')
    f.write(str(speeds2[i]) + '\t')
    f.write(str(b_zero*np.exp(-lambd*D_list2[i])) + '\t')
    f.write('\n')
    
f.close()


f = open('data/speed_R0_vs_D_linfit_pars.tsv', 'w')

header = '#parameter\tvalue\n'
f.write(header)

f.write('lambda\t' + str(lambda_tilde) + '\n')
f.write('beta0\t' + str(waves2[0].p.beta) + '\n')
f.write('alpha\t' + str(waves2[0].p.alpha) + '\n')
f.write('gamma\t' + str(waves2[0].p.gamma) + '\n')
f.write('Nh\t' + str(waves2[0].p.Nh) + '\n')
f.write('M\t' + str(waves2[0].p.M) + '\n')
f.write('cutoff\t' + str(waves2[0].p.cutoff) + '\n')

f.close()